In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils_data
from torch.autograd import Variable

from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

import pandas as pd
import numpy as np

import pickle
import json
import os
import warnings
import datetime

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"{torch.cuda.device_count()} GPU(s) available and will be used.\nGPU name(s): {[torch.cuda.get_device_name(n) for n in range(torch.cuda.device_count())]}")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

1 GPU(s) available and will be used.
GPU name(s): ['Tesla V100-PCIE-16GB']


In [3]:
RANDOM_STATE = 42

In [4]:
# Instantiate a dataframe on first dataset to get columns
nfl_df = pd.read_csv('../../dataset_generators/datasets/0_train_normalized.csv')
nfl_df

,Unnamed: 0,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,...,fantasy_points_r_avg_5_lag_1,fantasy_points_r_avg_8_lag_1,opp_team_dst_fp_r_avg_1_lag_1,opp_team_dst_fp_r_avg_3_lag_1,opp_team_dst_fp_r_avg_5_lag_1,opp_team_dst_fp_r_avg_8_lag_1,opp_team_dst_fp_rank_r_avg_1_lag_1,opp_team_dst_fp_rank_r_avg_3_lag_1,opp_team_dst_fp_rank_r_avg_5_lag_1,opp_team_dst_fp_rank_r_avg_8_lag_1
0,0,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,4,REG,...,0.123524,0.123524,0.162162,0.315405,0.315405,0.315405,0.903226,0.467742,0.467742,0.467742
1,1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,16,REG,...,0.114039,0.114039,0.405405,0.288378,0.302703,0.300541,0.306452,0.586129,0.525806,0.501935
2,2,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,17,REG,...,0.161678,0.161678,0.243243,0.270270,0.270270,0.260000,0.709677,0.634516,0.600000,0.586774
3,3,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2015,4,REG,...,0.197934,0.197934,0.054054,0.180270,0.180270,0.180270,0.983871,0.758065,0.758065,0.758065
4,4,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2015,11,REG,...,0.441821,0.441821,0.270270,0.270270,0.308108,0.337838,0.354839,0.457097,0.451613,0.445484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48770,48770,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,9,REG,...,0.155565,0.154089,0.189189,0.351351,0.329730,0.378378,0.741935,0.333226,0.367742,0.343226
48771,48771,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,12,REG,...,0.230185,0.196037,0.270270,0.207297,0.281081,0.300541,0.338710,0.645161,0.519355,0.480000
48772,48772,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,13,REG,...,0.216695,0.196037,0.324324,0.396486,0.351351,0.388649,0.354839,0.166774,0.309677,0.254194
48773,48773,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,14,REG,...,0.214165,0.196880,0.513514,0.459459,0.345946,0.364865,0.129032,0.177419,0.425806,0.377097


In [5]:
# Get normalized columns
y = ['fantasy_points']
X = [col for col in nfl_df.columns if col not in ('player_id'
                                                        , 'player_name'
                                                        , 'player_display_name'
                                                        , 'position','position_group'
                                                        , 'headshot_url'
                                                        , 'season'
                                                        , 'week'
                                                        , 'season_type'
                                                        , 'team'
                                                        , 'opponent_team'
                                                        , 'fantasy_points'
                                                        , 'Unnamed: 0')]
del nfl_df

In [6]:
nfl_df = pd.read_csv('../../dataset_generators/datasets/0_train_pca.csv')
nfl_df

,Unnamed: 0,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,...,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,fantasy_points
0,0,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,4,REG,...,-0.002146,0.005481,-0.052914,-0.047235,-0.032786,-0.021296,-0.043473,-0.430547,-0.039748,-0.20
1,1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,16,REG,...,0.004413,0.031793,-0.008367,-0.022133,0.001059,-0.050775,-0.044723,0.517862,-0.022523,7.04
2,2,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,17,REG,...,0.237716,-0.098981,-0.007436,0.217254,0.649703,3.571516,-0.360357,-0.098601,0.170117,9.40
3,3,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2015,4,REG,...,-0.040106,-0.041452,-0.099212,-0.047098,-0.119749,-2.255649,0.232299,-0.543449,0.557499,15.38
4,5,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2015,11,REG,...,-0.025855,0.029912,-0.029561,0.055859,0.202644,0.288437,-0.131221,0.460260,0.242840,13.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48770,60963,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,9,REG,...,0.027369,0.024282,-0.016385,0.064106,-0.041373,0.023893,-0.004612,-0.229813,0.021283,11.50
48771,60965,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,12,REG,...,0.040545,0.025307,-0.039539,0.200897,-0.080051,-0.005432,-0.048712,1.058756,-0.056765,1.80
48772,60966,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,13,REG,...,0.012423,0.106695,0.056796,0.100761,-0.037281,-0.044826,-0.593010,-0.323484,0.035053,2.00
48773,60967,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,14,REG,...,0.024793,0.091568,0.048286,0.080314,-0.008332,-0.039124,-0.354176,-0.783789,0.043358,1.90


In [7]:
# Get pca columns
y_pca = ['fantasy_points']
X_pca = [col for col in nfl_df.columns if col not in ('player_id'
                                                        , 'player_name'
                                                        , 'player_display_name'
                                                        , 'position','position_group'
                                                        , 'headshot_url'
                                                        , 'season'
                                                        , 'week'
                                                        , 'season_type'
                                                        , 'team'
                                                        , 'opponent_team'
                                                        , 'fantasy_points'
                                                        , 'Unnamed: 0')]
del nfl_df

In [8]:
# Function to get the 3 error metrics we decided to use
def get_errors(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, rmse, r2

In [9]:
# MLP Regressor but using PyTorch in order to use GPUs
# Code taken and tweaked from a question on pytorch forums: https://discuss.pytorch.org/t/sklearn-model-to-pytorch-model/33532
class relu_MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(relu_MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        out = self.fc3(x)
        return out

In [10]:
class tanh_MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(tanh_MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        out = self.fc3(x)
        return out

In [13]:
dataset_path = f'../../dataset_generators/datasets/0_test_pca.csv'
data = pd.read_csv(dataset_path)
X_test = data[X_pca]
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).cpu()
y_test = data[y_pca]
with open(f"./models/0_100-100_relu_100_0.01_pca.pickle", 'rb') as f:
    mlp = pickle.load(f)
    mlp.eval()
with torch.no_grad():
    y_pred = mlp(X_test_tensor)
    y_pred = y_pred.numpy()
mae, rmse, r2 = get_errors(y_test, y_pred)
print(f"MAE: {mae}\nRMSE: {rmse}\nR2: {r2}")

AcceleratorError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
